<a href="https://colab.research.google.com/github/mylethidiem/zero-to-hero/blob/main/preAIO25/external/VisionTransformer_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Vision Transformer**

In [ ]:
# Install packages
!pip install torchvision==0.17.2

In [ ]:
#
import torch
import torch.nn as nn
from torch.nn import functional as F

from torch.utils.data import DataLoader
import torchvision
import torchvision.transform as transform

In [ ]:
#?
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# LOAD DATA CIFAR10

What is [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html)?




In [ ]:
batch_size = 4

# Why need transform??
transform =  transform.Compose(
    [
        transform.Resize((224, 224)),
        transform.ToTensor(),
        transform.Normalize([(0.5, 0.5, 0.5), (0.5, 0.5, 0.5)]) #?
    ]
)

#download
test_dataset = torch.vision.dataset.CIFAR10(root='./data', train=False, transform=transform, download=True)

# split data train into val and train
train_dataset = torchvision.dataset.CIFAR10(root='./data', train=True, transform=transform, download=True)
train_subset_size = int(len(train_traindataset) * 0.9)
val_subset_size = len(train_traindataset) - train_subset_size
train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_subset_size,val_subset_size ])


# create dataloader contain batches of data to train(mini-batch)
train_loader = DataLoader(dataset=train_subset, batch_size=batch_size, shuffle=True, nums_worker=2)#?
val_loader = DataLoader(dataset=val_subset, batch_size=batch_size, shuffle=False, nums_worker=2)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, nums_worker=2)


In [ ]:
# Building Model
import torch.optim as optim

In [ ]:
# Functions
